In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#load the processed data
TRAIN_XX = np.load('TrainData_prep/TRAIN_XX_11var.npy')
TRAIN_YY = np.load('TrainData_prep/TRAIN_YY_11var.npy')

In [3]:
#reorder the axis for test_train_split
X = np.moveaxis(TRAIN_XX, 0, 2)
print(X.shape)
y = np.moveaxis(TRAIN_YY, 0, 2)
print(y.shape)

(128, 128, 3799, 11)
(128, 128, 3799, 1)


In [5]:
#Check the frequency of each class and the proportion of class 1
unique, counts = np.unique(y, return_counts=True)
print(unique[0], " :", counts[0])
print(unique[1], " :", counts[1])
toti = counts[0] + counts[1]
class1_prop = counts[1] / toti
print("tot  :", toti)
print("class 1 proportion: {:.2f} %".format(class1_prop * 100))

0.0  : 60800026
1.0  : 1442790
tot  : 62242816
class 1 proportion: 2.32 %


In [6]:
#Create an array which holds each image index which have landslide presence
result = np.where(y == 1)[2]
result_sort = np.sort(result)
result_sort = np.unique(result_sort)
print(result_sort[:10])

#Create an array "y_strat" which will hold the length of 3799 and a 0 indexed for each image without landslide and a 1 for each with landslide 
# (To enable stratify in test_train_split).
y_strat = np.zeros(3799)
for i, v in enumerate(result_sort):
    y_strat[v] = 1
print(y_strat[:10])

[ 0  2  3  4  5  6  7  8  9 10]
[1. 0. 1. 1. 1. 1. 1. 1. 1. 1.]


In [7]:
#Split the data into test, train while preserving the image shapes
from sklearn.model_selection import train_test_split
all_indices = list(range(3799))
train_ind, test_ind = train_test_split(all_indices,  random_state=0, shuffle=True, stratify=y_strat)

X_train_dummy = X[:,:,train_ind,:]
X_test_dummy = X[:,:,test_ind, :]

y_train_dummy = y[:,:,train_ind,:]
y_test_dummy = y[:,:,test_ind, :]

print(X_train_dummy.shape, X_test_dummy.shape)
print(y_train_dummy.shape, y_test_dummy.shape)

(128, 128, 2849, 11) (128, 128, 950, 11)
(128, 128, 2849, 1) (128, 128, 950, 1)


In [8]:
#Stack the X datasets into 2d arrays to fit the model
X_train = np.vstack((X_train_dummy[0]))
for i in X_train_dummy[1:]:
    X_train = np.concatenate((X_train,np.vstack((i))),axis=0)
print("new X_train shape: {}".format(X_train.shape))

X_test = np.vstack((X_test_dummy[0]))
for i in X_test_dummy[1:]:
    X_test = np.concatenate((X_test,np.vstack((i))),axis=0)
print("new X_test shape: {}".format(X_test.shape))

KeyboardInterrupt: 

In [ ]:
#Stack the y datasets into 1d arrays to fit the model
y_train = np.vstack((y_train_dummy[0]))
for i in y_train_dummy[1:]:
    y_train = np.concatenate((y_train,np.vstack((i))),axis=0)
y_train = y_train.reshape(-1)
print("new y_train shape: {}".format(y_train.shape))

y_test = np.vstack((y_test_dummy[0]))
for i in y_test_dummy [1:]:
    y_test = np.concatenate((y_test,np.vstack((i))),axis=0)
y_test = y_test.reshape(-1)
print("new y_test shape: {}".format(y_test.shape))

In [ ]:
#Due to the imbalanced distribution between 0's and 1's a RandomUnderSampler is performed.
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
# Randomly under sample the majority class (0's)
rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus= rus.fit_resample(X_train, y_train)

In [ ]:
#Verify the resampled distribution between the two classes
unique, counts = np.unique(y_train_rus, return_counts=True)
print(unique[0], " :", counts[0])
print(unique[1], " :", counts[1])
toti = counts[0] + counts[1]
class1_prop = counts[1] / toti
print("tot  :", toti)
print("class 1 proportion: {:.2f} %".format(class1_prop * 100))

In [ ]:
#Check the shape of the stacked train/test datasets
print("X_train_rus shape:", X_train_rus.shape)
print("y_train_rus shape:", y_train_rus.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

In [ ]:
#Instantiate Random Forest Classifier with 100 trees
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100, random_state=0, verbose=3,n_jobs=-1)

In [ ]:
#Fit the model
rfc.fit(X_train_rus, y_train_rus)

In [ ]:
#Predict
y_pred = rfc.predict(X_test)

In [ ]:
#Evaluate the prediction of the test dataset, first
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print("Confusion Matrix:\n")
print("True Positive: ", confusion_matrix(y_test,y_pred)[0][0])
print("False Negative: ", confusion_matrix(y_test,y_pred)[0][1])
print("False Positive: ", confusion_matrix(y_test,y_pred)[1][0])
print("True Negative: ", confusion_matrix(y_test,y_pred)[1][1])
print("\nClassification Report: \n")
print(classification_report(y_test,y_pred))

In [ ]:
#Due to the skewness between precision and recall the threshold is set to 0.95, it worsens precision but improves recall
threshold = 0.95
predicted_proba = rfc.predict_proba(X_test)
y_pred = (predicted_proba [:,1] >= threshold).astype('int')

In [ ]:
#Evaluate the prediction of the test dataset, second
#With a changed threshold we receive a better f1-score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print("Confusion Matrix:\n")
print("True Positive: ", confusion_matrix(y_test,y_pred)[0][0])
print("False Negative: ", confusion_matrix(y_test,y_pred)[0][1])
print("False Positive: ", confusion_matrix(y_test,y_pred)[1][0])
print("True Negative: ", confusion_matrix(y_test,y_pred)[1][1])
print("\nClassification Report: \n")
print(classification_report(y_test,y_pred))

In [ ]:
#Create a list containing each feature name
feature_names = ['RED', 'GREEN', 'BLUE', 'NDVI', 'SLOPE', 'ELEVATION', 'NDMI', 'GNDVI', 'BAND 10', 'BRIGHTNESS', 'BSI']
print(len(feature_names))

#Extract feature importance and std
importances = rfc.feature_importances_
std = np.std([tree.feature_importances_ for tree in rfc.estimators_], axis=0)
forest_importances = pd.Series(importances, index=feature_names)

#Plot the feature importance
fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

In [ ]:
#Restore the predicted data into images of size 128x128
y_pred_reshaped = y_pred.reshape(128, 128, 950, 1)
print(y_pred_reshaped.shape)

In [ ]:
#Plot the most important features, the ground truth and the predicted output of an example image
import matplotlib
import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=2, ncols=5, sharex=True, sharey=True, figsize=(15,7))
ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9, ax10 = axes.flatten()

ax1.set_title("RGB", fontweight='bold', fontsize='16')
ax1.imshow(X_test_dummy[:,:,40,0:3])

ax2.set_title("NDVI", fontweight='bold', fontsize='16')
ax2.imshow(X_test_dummy[:,:,40,3], cmap='gist_rainbow')

ax3.set_title("SLOPE", fontweight='bold', fontsize='16')
ax3.imshow(X_test_dummy[:,:,40,4], cmap='gist_rainbow')

ax4.set_title("ELEVATION", fontweight='bold', fontsize='16')
ax4.imshow(X_test_dummy[:,:,40,5], cmap='gist_rainbow')

ax5.set_title("NDMI", fontweight='bold', fontsize='16')
ax5.imshow(X_test_dummy[:,:,40,6], cmap='coolwarm_r')

ax6.set_title("GNDVI", fontweight='bold', fontsize='16')
ax6.imshow(X_test_dummy[:,:,40,7], cmap='gist_rainbow')

ax7.set_title("Brightness", fontweight='bold', fontsize='16')
ax7.imshow(X_test_dummy[:,:,40,9], cmap='gist_rainbow')

ax8.set_title("BSI", fontweight='bold', fontsize='16')
ax8.imshow(X_test_dummy[:,:,40,10], cmap='gist_rainbow')

ax9.set_title("Ground Truth", fontweight='bold', fontsize='16')
ax9.imshow(y_test_dummy[:,:,40,:])

ax10.set_title("Predicted", fontweight='bold', fontsize='16')
ax10.imshow(y_pred_reshaped[:,:,40,:])
plt.show()